In [48]:
# Se cargan las librerías y tabla de datos
import pandas as pd
import plotly.express as px
from datetime import datetime
from plotly import graph_objects as go
import os
# os.getcwd()
os.chdir('/home/clautc/Proyectos_/Estudios/Estudio_crianzas_Chile')
from plotly.subplots import make_subplots
ruta = 'data/registros_estadisticos/df_trabajo.csv'
df = pd.read_csv(ruta)
df.drop(columns='Unnamed: 0', inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4513701 entries, 0 to 4513700
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ano_nac      int64  
 1   activ_madre  float64
 2   activ_padre  float64
 3   ocupa_padre  object 
 4   ocupa_madre  object 
 5   categ_padre  int64  
 6   categ_madre  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 241.1+ MB


In [2]:
df.info()
df.activ_padre.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4513701 entries, 0 to 4513700
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ano_nac      int64  
 1   activ_madre  float64
 2   activ_padre  float64
 3   ocupa_padre  object 
 4   ocupa_madre  object 
 5   categ_padre  int64  
 6   categ_madre  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 241.1+ MB


array([ 1.,  2.,  0., nan,  3.])

In [49]:
actividad = {
    0.0: 'Inactivas',
    1.0: 'Activas',
    2.0: 'Cesantes',
    9.0: 'Ignorado',
    8.0: 'Otros'
}

df.loc[df.activ_madre.isna(), 'activ_madre'] = 8.0
df.loc[df.activ_padre.isna(), 'activ_padre'] = 8.0
df.loc[df.activ_madre==9.0, 'activ_madre'] = 8.0
df.loc[df.activ_padre==9.0, 'activ_padre'] = 8.0
df['caem'] = df.activ_madre.apply(lambda x: actividad.get(x))
df['caeh'] = df.activ_padre.apply(lambda x: actividad.get(x))
df['aniof'] = pd.to_datetime(df.ano_nac, format='%Y')
df['anio'] = df.aniof.dt.year
# df = df[~((df.anio == 2007) | (df.anio == 2015))]
# df.loc[df.anio==2015]
gr_anio = df.groupby('anio')
gr_act_m = df.groupby(['anio', 'caem', 'activ_madre'])
gr_act_h = df.groupby(['anio', 'aniof','caeh', 'activ_padre'])
df['tot'] = gr_anio['caem'].transform('count')
df['actm'] = gr_act_m['caem'].transform('count')
gr_act_m = df.groupby(['anio', 'aniof', 'caem', 'activ_madre'])

df['acth'] = gr_act_h['caem'].transform('count')

df['propm'] = df.actm/df.tot*100
df['proph'] = df.acth/df.tot*100

activ_mama = df[['anio', 'aniof', 'caem', 'activ_madre', 'propm']].drop_duplicates().reset_index(drop=True)
activ_papa = df[['anio', 'aniof', 'caeh', 'activ_padre', 'proph']].drop_duplicates().reset_index(drop=True)

In [53]:
activ_mama = activ_mama.sort_values('anio').reset_index(drop=True)

# activ_mama.columns
activ_mama.aniof.to_list()

[Timestamp('2001-01-01 00:00:00'),
 Timestamp('2001-01-01 00:00:00'),
 Timestamp('2001-01-01 00:00:00'),
 Timestamp('2002-01-01 00:00:00'),
 Timestamp('2002-01-01 00:00:00'),
 Timestamp('2002-01-01 00:00:00'),
 Timestamp('2002-01-01 00:00:00'),
 Timestamp('2003-01-01 00:00:00'),
 Timestamp('2003-01-01 00:00:00'),
 Timestamp('2003-01-01 00:00:00'),
 Timestamp('2004-01-01 00:00:00'),
 Timestamp('2004-01-01 00:00:00'),
 Timestamp('2004-01-01 00:00:00'),
 Timestamp('2005-01-01 00:00:00'),
 Timestamp('2005-01-01 00:00:00'),
 Timestamp('2005-01-01 00:00:00'),
 Timestamp('2006-01-01 00:00:00'),
 Timestamp('2006-01-01 00:00:00'),
 Timestamp('2006-01-01 00:00:00'),
 Timestamp('2006-01-01 00:00:00'),
 Timestamp('2007-01-01 00:00:00'),
 Timestamp('2007-01-01 00:00:00'),
 Timestamp('2007-01-01 00:00:00'),
 Timestamp('2008-01-01 00:00:00'),
 Timestamp('2008-01-01 00:00:00'),
 Timestamp('2008-01-01 00:00:00'),
 Timestamp('2008-01-01 00:00:00'),
 Timestamp('2009-01-01 00:00:00'),
 Timestamp('2009-01-

In [66]:
fig = go.Figure()
for c in activ_mama.caem.unique():
    # texto = str(round(activ_mama.loc[activ_mama.caem==c, 'propm'].to_list()[0]))
    fig.add_trace(go.Bar(x=activ_mama.loc[activ_mama.caem==c, 'aniof'],
                  y=activ_mama.loc[activ_mama.caem==c, 'propm'],
                         xperiod='M12',
                         name = c,
                         text= activ_mama.loc[activ_mama.caem==c, 'propm'],
                         textposition='auto',
                         texttemplate="%{text:.1f}"
                ),)

fig.update_layout(barmode='relative',
                  template='none')
fig.update_xaxes(
    rangebreaks=[
        dict(values=['2007-01-01 00:00:00', '2015-01-01 00:00:00'])
    ]
)

fig.update_yaxes(range=[0,100])
fig.write_image('resultados/img/04_caem.jpeg')
fig.show()



In [68]:
fig = go.Figure()
for c in activ_papa.caeh.unique():
    # texto = str(round(activ_mama.loc[activ_mama.caem==c, 'propm'].to_list()[0]))
    fig.add_trace(go.Bar(x=activ_papa.loc[activ_papa.caeh==c, 'aniof'],
                         y=activ_papa.loc[activ_papa.caeh==c, 'proph'],
                         xperiod='M12',
                         name = c,
                         text= activ_papa.loc[activ_papa.caeh==c, 'proph'],
                         textposition='auto',
                         texttemplate="%{text:.1f}"
                         ),)

fig.update_layout(barmode='relative',
                  template='none')
fig.update_xaxes(
    rangebreaks=[
        dict(values=['2007-01-01 00:00:00', '2015-01-01 00:00:00'])
    ]
)

fig.update_yaxes(range=[0,100])
fig.write_image('resultados/img/04_caep.jpeg')
fig.show()

AttributeError: 'DataFrame' object has no attribute 'caep'